In [ ]:
%load_ext autoreload
%autoreload 2
from main import do
from main import do_v2
import numpy as np
from sklearn.model_selection import train_test_split
from preprocess import contents2count, contents2count_v2
from sklearn.metrics.pairwise import cosine_similarity
from preprocess import AE2
from tqdm import tqdm
from sklearn.metrics import f1_score
import pandas as pd 
from collections import Counter
import random
from glob import glob
import os


In [ ]:
def hex2ascii(x):
    output = ''
    for i in range(0,len(x),2):
        tmp = str(x[i:i+2])
        output+=chr(int(tmp,16))
    return output

In [ ]:
def read_opendata(path):
    datas_ = []
    for paths in glob(os.path.join(path,'*.txt')):
        datas = []
        with open(paths) as f:
            event=''
            tmp1=''
            tmp2=''
            for i in f.readlines():
                if (i[:3]=='GET' )| (i[:4]=='POST'):
                    datas.append(event)
                    event = i
                else:
                    event+=i
        datas_.append(datas[1:])



    labels = list(map(lambda x: np.zeros(len(x)), datas_))
    labels[0] +=1

    del datas
    tr1,te1 =  train_test_split(datas_[0],test_size=0.2,shuffle=True,random_state=666)
    tr2,te2 =  train_test_split(datas_[1],test_size=0.2,shuffle=True,random_state=666)

    train = tr1 + tr2
    train_label = np.array([0]*len(tr1) + [1]*len(tr2))
    test = te1 + te2
    test_label = np.array([0]*len(te1) + [1]*len(te2))

    train = list(map(lambda x: "\n".join([x.splitlines()[0],x.splitlines()[-2]]) , train))
    test_ = list(map(lambda x: "\n".join([x.splitlines()[0],x.splitlines()[-2]]) , test))
    return train, test_ ,train_label,test_label

In [ ]:
def find_centroid(res,test,th):
    
    cluster_center = {}
    for i in res[1]:
        cluster_center[i] = np.mean(res[1][i],axis=0)

    ress = []
    keys = list(cluster_center.keys())
    cluster_center_vectors = np.array(list(cluster_center.values()))

    for ev in test:
        ev = ev.reshape(1,-1)
        sim = cosine_similarity(ev,cluster_center_vectors)
        sim_max = np.max(sim)
        if sim_max<th:
            ress.append(-1)
        else:
            ress.append(keys[np.argmax(sim)])

    return np.array(ress)

In [ ]:
def random_pick(x,n):
    if int(x[0]+x[1]-1) <n:
        pa = n
    else:
        pa = int(x[0]+x[1]-1)
    p = np.random.choice(pa, n,replace=False)
    picked  = list(map(lambda y: 0 if x[0]>y  else 1 , p))
    if len(np.unique(picked)) ==2:
        output = -1
    else:
        output = picked[0]
    return output

In [ ]:
def clustering_labeling(res,train_label,n):
    df = pd.DataFrame([res,train_label])
    df = df.transpose()
    df.columns = ['pred','label']
    values = df.groupby('pred').value_counts().reset_index()
    card = df.groupby('pred').value_counts().reset_index().groupby('pred').nunique().reset_index()
    card.columns = ['pred','label','card']
    card[0]=0
    card[1]=0

    for _,i in values.iterrows():
        card.loc[card.pred == i['pred'],i.label] = i[0]

    card['ratio'] = card[1]/(card[0] + card[1])

    card['random_pick'] = card.apply(random_pick,axis=1,n=n)
    cluster_label = card.set_index('pred').random_pick.to_dict()
    return card,cluster_label
 

In [ ]:
def testing(card,pred,cluster_label):
    pred_ = np.array(list(map(lambda x: cluster_label[x],pred)), dtype=np.int32)
    return pred_

In [ ]:
def read_private_dataset(path):
    df= pd.read_csv(path)
    df = df[~df.encrypt]
    idx = df.index
    train_idx, test_idx = train_test_split(idx,test_size=0.2,shuffle=False)
    df['decoded'] = df.payload.apply(hex2ascii)
    train_ = df.loc[train_idx]
    test_ = df.loc[test_idx]


    train_label = df.loc[train_idx, 'analyResult'].to_numpy()
    test_label = df.loc[test_idx, 'analyResult'].to_numpy()

    return train_,test_

아래는 오픈 데이터셋 실험 코드


In [ ]:

th = 0.95
n = 1

# train, test_ ,train_label,test_label =  read_opendata('./dataset/')

res = do_v2(train,512,4,th,th,3)

test = res[-1].transform(test_)

cluster_center = find_centroid(res,test,th)

pred = find_centroid(res,test,th)

card,cluster_label = clustering_labeling(res[0],train_label,n)

pred_ = testing(card,pred,cluster_label)
coverage_index = pred!=-1

f1  = f1_score(test_label[coverage_index],pred_[coverage_index])

coverage = coverage_index.sum()/coverage_index.shape[0]

card,cluster_label = clustering_labeling(res[0],train_label,n)

pred_ = testing(card,pred,cluster_label)
coverage_index = pred!=-1

f1  = f1_score(test_label[coverage_index],pred_[coverage_index])

coverage = coverage_index.sum()/coverage_index.shape[0]

아래는 kisti 데이터셋 실험 코드

In [ ]:

# public datase

th_ = np.arange(0.1,0.99,0.05)

res_={}

train_, test_ = read_private_dataset('./dataset/public.csv')

for th in th_:
    n = 1
    min_count = 3
    vector_size= 512
    # train, test_ ,train_label,test_label =  read_opendata('./dataset/')

    pred_all, coverage_all = [[],[]],[0,0]

    no_iip=0
    remain_out = 0
    for iip in tqdm(train_.machine.unique()):
        for na in train_.detectName.unique():
            train = train_[(train_.machine==iip)&(train_.detectName==na)]
            
            test = test_[(test_.machine==iip)&(test_.detectName==na)]

            train_label = train.analyResult.to_numpy()
            test_label = test.analyResult.to_numpy()

            train_decoded = train.decoded
            test_decoded = test.decoded

            if len(test)==0:
                continue
            
            if len(train)<=min_count:
                no_iip+=len(test)
                continue

            res = do_v2(train_decoded,vector_size,4,th,th,min_count)

            test = res[-1].transform(test_decoded)

            cluster_center = find_centroid(res,test,th)

            pred = find_centroid(res,test,th)

            card,cluster_label = clustering_labeling(res[0],train_label,n)
            cluster_label[-1]=-1
            pred_ = testing(card,pred,cluster_label)
            coverage_index = pred_!=-1
            remain_out += np.sum(pred==-1)
            coverage = (coverage_index.sum(),coverage_index.shape[0])
            
            pred_all[0].extend(test_label[coverage_index])
            pred_all[1].extend(pred_[coverage_index])
            coverage_all[0]+=coverage_index.sum()
            coverage_all[1]+=coverage_index.shape[0]
    
    res_[th] = (pred_all,coverage_all,no_iip,remain_out)

f1_ = []
cover = []
for th in res_:
    f1_.append(f1_score(res_[th][0][0],res_[th][0][1]))
    cover.append(res_[th][1][0]/res_[th][1][1])


In [ ]:
[th,f1,coverage]